In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pySparkSetup').getOrCreate()

24/05/31 14:29:17 WARN Utils: Your hostname, Ugurs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.30 instead (on interface en0)
24/05/31 14:29:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/31 14:29:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/31 14:29:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
weather = spark.read.parquet("/Users/ugurkalkavan/Downloads/m06sparkbasics/weather")

In [3]:
spark.conf.get("spark.sql.parquet.filterPushdown")

'true'

In [4]:
spark.conf.get("spark.sql.parquet.aggregatePushdown")

'false'

In [26]:
from pyspark.sql.functions import approx_count_distinct

def get_high_cardinality_columns_ratio(df, threshold_ratio):
    """DataFrame'deki her bir sütunun cardinality'sini hesaplar ve cardinality'si
    belirli bir eşik değerinin üzerinde olan sütunları döndürür."""
    column_list = df.columns
    high_cardinality_columns = []
    total_count = df.count()

    for column in column_list:
        count = df.agg(approx_count_distinct(column)).collect()[0][0]
        ratio = count / total_count
        if ratio > threshold_ratio:
            high_cardinality_columns.append((column, count, ratio))

    return high_cardinality_columns



In [24]:
# Eşik değerini belirleyin. Bu, cardinality oranının ne kadar yüksek kabul edildiğini belirlemek için kullanılır.
# Bu değer, veri kümenize, uygulamanıza ve performansınıza bağlı olarak ayarlanmalıdır.
threshold_ratio = 0.75

# DataFrame'deki her bir sütun için cardinality oranını hesapla ve yüksek cardinality'li sütunları bul
high_cardinality_columns = get_high_cardinality_columns_ratio(weather, threshold_ratio)

for column, count, ratio in high_cardinality_columns:
    print(f"Kolon: {column}, Cardinality: {count}, Oran: {ratio}")

In [30]:
def get_column_value_counts(df, column):
    """Belirli bir sütundaki benzersiz değerlerin dağılımını ve bunların ne sıklıkta göründüğünü belirler."""
    value_counts = df.groupBy(column).count().orderBy('count', ascending=False)
    return value_counts

In [32]:
# Örneğin 'p_date' sütunu için değer sayılarını alalım
value_counts = get_column_value_counts(weather, 'year')

# Sonuçları yazdıralım
value_counts.show()

+----+--------+
|year|   count|
+----+--------+
|2017|75061598|
|2016|37333145|
+----+--------+



In [36]:
def get_all_column_value_counts(df):
    """Verilen DataFrame'deki tüm sütunlardaki benzersiz değerlerin dağılımını ve bunların geliş sıklığını belirler."""
    column_list = df.columns
    all_column_counts = {}

    for column in column_list:
        value_counts = df.groupBy(column).count().orderBy('count', ascending=False)
        all_column_counts[column] = value_counts
    
    return all_column_counts

In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql import Row
from pyspark.sql.functions import approx_count_distinct


def get_top_value_counts(df):
    """Verilen DataFrame içerisindeki tüm sütunlardaki en yüksek değerli kayıtları DataFrame olarak döndürür."""
    top_value_counts = []
    spark.sparkContext.setJobDescription('df count')  # Setting Job description
    total_rows = df.count()

    for column in df.columns:
        # Null kontrolü
        spark.sparkContext.setJobDescription(f'df group by count sort {column}')  # Setting Job description
        col_count = df.groupBy(column).count().sort(desc("count"))
        
        spark.sparkContext.setJobDescription(f'df approx distinct count {column}')  # Setting Job description
        approx_distinct_value = df.agg(approx_count_distinct(column)).collect()[0][0]
        
        spark.sparkContext.setJobDescription(f'df head {column}')  # Setting Job description
        if col_count.head() is not None:
            spark.sparkContext.setJobDescription(f'df first {column}')  # Setting Job description
            row = col_count.first()
            ratio = round((row[1] / total_rows) * 100, 2)  # calculate the ratio
            top_value_counts.append(Row(column, row[0], row[1], ratio, approx_distinct_value))

    result_df = spark.createDataFrame(top_value_counts, ["Column", "Value", "Count", "Ratio", "Approx_Distinct_Value"])

    return result_df.sort(desc("Count"))

In [11]:
spark.sparkContext.setJobDescription('without agg')  # Setting Job description

In [27]:
# DataFrame'i oluşturalım ve yazdıralım
spark.conf.set("spark.sql.parquet.aggregatePushdown", True)
top_value_df = get_top_value_counts(weather)
top_value_df.show()

24/05/31 15:26:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/31 15:28:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/31 15:30:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/31 15:31:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+----------+----------+---------+-----+---------------------+
|    Column|     Value|    Count|Ratio|Approx_Distinct_Value|
+----------+----------+---------+-----+---------------------+
|      year|      2018|450787193|12.51|                    8|
|     month|        12|305168464| 8.47|                   12|
|       day|        29|118024836| 3.27|                   32|
| wthr_date|2017-08-29| 59064864| 1.64|                   92|
|avg_tmpr_c|      28.4| 18089988|  0.5|                  884|
|avg_tmpr_f|      83.1|  9078272| 0.25|                 1602|
|       lat|   66.7461|  4773296| 0.13|               333940|
|       lng|   19.2272|  1671240| 0.05|               324657|
+----------+----------+---------+-----+---------------------+



In [14]:
spark.sparkContext.setJobDescription('with agg')  # Setting Job description

In [15]:
spark.conf.set("spark.sql.parquet.aggregatePushdown", True)

In [16]:
# DataFrame'i oluşturalım ve yazdıralım
top_value_df = get_top_value_counts(weather)
top_value_df.show()

24/05/31 15:03:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/31 15:07:26 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+----------+----------+---------+-----+---------------------+
|    Column|     Value|    Count|Ratio|Approx_Distinct_Value|
+----------+----------+---------+-----+---------------------+
|      year|      2018|450787193|12.51|                    8|
|     month|        12|305168464| 8.47|                   12|
|       day|        29|118024836| 3.27|                   32|
| wthr_date|2017-08-29| 59064864| 1.64|                   92|
|avg_tmpr_c|      28.4| 18089988|  0.5|                  884|
|avg_tmpr_f|      83.1|  9078272| 0.25|                 1602|
|       lat|   66.7461|  4773296| 0.13|               333940|
|       lng|   19.2272|  1671240| 0.05|               324657|
+----------+----------+---------+-----+---------------------+



In [17]:
spark

In [30]:
weather.select("_metadata.file_path", "_metadata.file_name","_metadata.file_size", "_metadata.file_modification_time").show(2,False)


+-------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------+---------+----------------------+
|file_path                                                                                                                                              |file_name                                                          |file_size|file_modification_time|
+-------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------+---------+----------------------+
|file:/Users/ugurkalkavan/Downloads/m06sparkbasics/weather/year=2017/month=08/day=29/part-00088-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet|part-00088-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet|6098626  |2023

In [46]:
weather.select("_metadata.file_block_length").show(1,False)


+-----------------+
|file_block_length|
+-----------------+
|6098626          |
+-----------------+
only showing top 1 row



In [11]:
weather

DataFrame[lng: double, lat: double, avg_tmpr_f: double, avg_tmpr_c: double, wthr_date: string, year: int, month: int, day: int]

In [23]:
spark.conf.set("spark.sql.parquet.aggregatePushdown", False)
spark.sparkContext.setJobDescription('without agg day')  # Setting Job description
weather.groupBy("day").count().sort(desc("count")).show()
weather.agg({"day": "count"}).show()

+---+-------+
|day|  count|
+---+-------+
| 28|3665331|
| 27|3665331|
| 26|3665331|
|  9|3665331|
|  8|3665331|
| 23|3665331|
|  7|3665331|
| 10|3665331|
| 25|3665331|
| 24|3665331|
| 29|3665331|
| 30|3665331|
| 12|3665331|
| 22|3665331|
|  1|3665331|
| 13|3665331|
|  6|3665331|
| 20|3665331|
|  5|3665331|
| 15|3665331|
+---+-------+
only showing top 20 rows

+----------+
|count(day)|
+----------+
| 112394743|
+----------+



In [14]:
spark.conf.set("spark.sql.parquet.aggregatePushdown", True)
spark.sparkContext.setJobDescription('with agg day')  # Setting Job description
weather.groupBy("day").count().sort(desc("count")).show()

+---+-------+
|day|  count|
+---+-------+
| 28|3665331|
| 27|3665331|
| 26|3665331|
|  9|3665331|
|  8|3665331|
| 23|3665331|
|  7|3665331|
| 10|3665331|
| 25|3665331|
| 24|3665331|
| 29|3665331|
| 30|3665331|
| 12|3665331|
| 22|3665331|
|  1|3665331|
| 13|3665331|
|  6|3665331|
| 20|3665331|
|  5|3665331|
| 15|3665331|
+---+-------+
only showing top 20 rows



In [16]:
spark.conf.set("spark.sql.parquet.aggregatePushdown", False)
spark.sparkContext.setJobDescription('without agg wthr_date')  # Setting Job description
weather.groupBy("wthr_date").count().sort(desc("count")).show()

+----------+-------+
| wthr_date|  count|
+----------+-------+
|2017-08-31|1230518|
|2017-08-27|1230518|
|2017-08-26|1230518|
|2017-09-10|1230518|
|2017-08-23|1230518|
|2017-08-25|1230518|
|2017-09-09|1230518|
|2017-09-08|1230518|
|2017-08-29|1230518|
|2017-08-30|1230518|
|2017-08-28|1230518|
|2017-09-07|1230518|
|2017-08-24|1230518|
|2017-09-12|1230518|
|2017-08-10|1230518|
|2017-09-06|1230518|
|2017-09-13|1230518|
|2017-08-22|1230518|
|2017-09-01|1230518|
|2017-08-15|1230518|
+----------+-------+
only showing top 20 rows



In [18]:
spark.conf.set("spark.sql.parquet.aggregatePushdown", True)
spark.sparkContext.setJobDescription('with agg wthr_date')  # Setting Job description
weather.groupBy("wthr_date").count().sort(desc("count")).show()

+----------+-------+
| wthr_date|  count|
+----------+-------+
|2017-08-31|1230518|
|2017-08-27|1230518|
|2017-08-26|1230518|
|2017-09-10|1230518|
|2017-08-23|1230518|
|2017-08-25|1230518|
|2017-09-09|1230518|
|2017-09-08|1230518|
|2017-08-29|1230518|
|2017-08-30|1230518|
|2017-08-28|1230518|
|2017-09-07|1230518|
|2017-08-24|1230518|
|2017-09-12|1230518|
|2017-08-10|1230518|
|2017-09-06|1230518|
|2017-09-13|1230518|
|2017-08-22|1230518|
|2017-09-01|1230518|
|2017-08-15|1230518|
+----------+-------+
only showing top 20 rows



In [20]:
spark